In [6]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

In [7]:
cid ='1fee343b24a04165a0c05a14508d3929'
secret ='4ed932cb67bb4b879456cf0032d751ef'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [8]:
#insert the URI as a string into the function
def get_album_tracks(uri_info):
    uri = []
    track = []
    duration = []
    explicit = []
    track_number = []
    one = sp.album_tracks(uri_info, limit=50, offset=0, market='PH')
    df1 = pd.DataFrame(one)
    
    for i, x in df1['items'].items():
        uri.append(x['uri'])
        track.append(x['name'])
        duration.append(x['duration_ms'])
        explicit.append(x['explicit'])
        track_number.append(x['track_number'])
    
    df2 = pd.DataFrame({
    'uri':uri,
    'track':track,
    'duration_ms':duration,
    'explicit':explicit,
    'track_number':track_number})
    
    return df2

In [9]:
#insert output dataframe from the get_album_tracks function
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    for i in df['uri']:
        for x in sp.audio_features(tracks=[i]):
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [10]:
def merge_frames(df1, df2):
    df3 = df1.merge(df2, left_index= True, right_index= True)
    return df3

In [11]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1

In [29]:
df1 = get_album_tracks("https://open.spotify.com/album/3SpBlxme9WbeQdI9kx7KAV?si=uLAMGRtXS66izabItbjekA&dl_branch=1")
# https://open.spotify.com/album/0HYAcVSUVkfxzHKgHu6DqR?si=Wt5z1yWESVGDWlmtoFfocg&dl_branch=1 kamikazee album
# https://open.spotify.com/playlist/37i9dQZF1DX4VkTBdvsYXu?si=8fc51c15abc6479d drake playlist

In [30]:
df1 = pd.DataFrame(df1["track"])

In [31]:
# clara_tracks_meta = get_track_info(clara_tracks)

In [32]:
# clara_tracks_meta

In [33]:
# df1 = merge_frames(clara_tracks, clara_tracks_meta)

In [34]:
df1

,track
0,Champagne Poetry
1,Papi’s Home
2,Girls Want Girls (with Lil Baby)
3,In The Bible (with Lil Durk & Giveon)
4,Love All (with JAY-Z)
5,Fair Trade (with Travis Scott)
6,Way 2 Sexy (with Future & Young Thug)
7,TSU
8,N 2 Deep
9,Pipe Down


In [36]:
df2 = lyrics_onto_frame(df1, 'Drake')

In [37]:
df2

,track,lyrics
0,Champagne Poetry,"\n\n[Part I]\n\n[Intro]\nI love you, I love yo..."
1,Papi’s Home,None
2,Girls Want Girls (with Lil Baby),None
3,In The Bible (with Lil Durk & Giveon),None
4,Love All (with JAY-Z),None
5,Fair Trade (with Travis Scott),None
6,Way 2 Sexy (with Future & Young Thug),None
7,TSU,"\n\n[Intro: OG Ron C]\nYeah, we back\nWhat's u..."
8,N 2 Deep,None
9,Pipe Down,\n\n[Intro]\n(I'm working on dying)\nSaid you ...


In [ ]:
df2

In [ ]:
#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1